##### # Import neccessary modules
1 for matrix caculation, 2 for Deep Learning

In [1]:
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
from module.input_parser_old import input_parser
from module import in_fun
from module.torch import BinaryClassifier
from module import pl_module
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers


beacon_list = [f"beacon{idx + 1}" for idx in range(9)]

%matplotlib widget
%load_ext tensorboard
%tensorboard --logdir lightning_logs

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Load Data
python에서 csv파일을 읽어올때, 숫자만 있는것이 아닌 문자열이 포함된 데이터를 읽을 경우 느려지는 것 확인

In [3]:
data_path = "data/train/infra_free/"
room_list = [room + '/' for room in os.listdir(data_path)]
room_data_list = []
for room in room_list:
    file_list = [file for file in os.listdir(data_path + room) if file.endswith(".csv")]
    cur_room_file = []
    for file in file_list:
        cur_room_file.append(pd.read_csv(data_path + room + file, header=None).to_numpy())
    room_data_list.append(np.concatenate((cur_room_file), axis=0))

In [4]:
for room_data in room_data_list:
    print(np.shape(room_data))

(26, 101)
(22, 101)
(22, 101)
(22, 101)


# Data 만들기
같은 방 데이터 끼리 매칭시키고, 다른 방 데이터 끼리 매칭시키고

In [5]:
matched_same_ble = []
for room_data in room_data_list:
    matched_same_ble.append(in_fun.train_matcher(room_data[:,1:41], room_data[:,1:41]))
same_diff_ble = np.concatenate([same[0] for same in matched_same_ble])
same_each_ble = np.concatenate([same[1] for same in matched_same_ble])
print(np.shape(same_each_ble))

(2128, 80)


In [6]:
matched_diff_ble = []
for idx in range(len(room_data_list)):
    if idx == 0:
        matched_diff_ble.append(in_fun.train_matcher(room_data_list[-1][:,1:41], room_data_list[idx][:,1:41]))
    else:
        matched_diff_ble.append(in_fun.train_matcher(room_data_list[idx-1][:,1:41], room_data_list[idx][:,1:41]))
    
diff_diff_ble = np.concatenate([diff[0] for diff in matched_diff_ble])
diff_each_ble = np.concatenate([diff[1] for diff in matched_diff_ble])
print(np.shape(diff_each_ble))

(2112, 80)


In [7]:
matched_same_wifi = []
for room_data in room_data_list:
    matched_same_wifi.append(in_fun.train_matcher(room_data[:,41:81], room_data[:,41:81]))
same_diff_wifi = np.concatenate([same[0] for same in matched_same_wifi])
same_each_wifi = np.concatenate([same[1] for same in matched_same_wifi])
print(np.shape(same_each_wifi))

(2128, 80)


In [8]:
matched_diff_wifi = []
for idx in range(len(room_data_list)):
    if idx == 0:
        matched_diff_wifi.append(in_fun.train_matcher(room_data_list[-1][:,41:81], room_data_list[idx][:,41:81]))
    else:
        matched_diff_wifi.append(in_fun.train_matcher(room_data_list[idx-1][:,41:81], room_data_list[idx][:,41:81]))
    
diff_diff_wifi = np.concatenate([diff[0] for diff in matched_diff_wifi])
diff_each_wifi = np.concatenate([diff[1] for diff in matched_diff_wifi])
print(np.shape(diff_each_ble))

(2112, 80)


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

cuda:0


In [10]:
print(len(same_each_ble) / (len(same_each_ble) + len(diff_each_ble)))

0.5018867924528302


# Loading data for nn

In [11]:
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data.dataset import random_split

x_tensor_diff = torch.from_numpy(np.concatenate((same_diff_ble, diff_diff_ble))).float()
x_tensor_each = torch.from_numpy(np.concatenate((same_each_ble, diff_each_ble))).float()

y = np.concatenate((np.zeros([len(same_each_ble), 1]) + 1, np.zeros([len(diff_each_ble), 1])))
y_tensor = torch.from_numpy(y).float()

dataset_diff = TensorDataset(x_tensor_diff, y_tensor)
dataset_each = TensorDataset(x_tensor_each, y_tensor)

train_len_diff = len(dataset_diff) // 5 * 4
train_len_each = len(dataset_each) // 5 * 4
val_len_diff = len(dataset_diff) - train_len_diff
val_len_each  = len(dataset_each) - train_len_each

train_dataset_diff, val_dataset_diff = random_split(dataset_diff, [train_len_diff, val_len_diff])
train_dataset_each, val_dataset_each = random_split(dataset_each, [train_len_each, val_len_each])

train_loader_diff = DataLoader(dataset=train_dataset_diff, batch_size=32, shuffle=True)
train_loader_each = DataLoader(dataset=train_dataset_each, batch_size=32, shuffle=True)
val_loader_diff = DataLoader(dataset = val_dataset_diff, batch_size = val_len_diff)
val_loader_each = DataLoader(dataset = val_dataset_each, batch_size = val_len_each)

In [12]:
annModel = pl_module.AnnModel(input_size = 40)
binaryClassifier = pl_module.BinaryClassifier(annModel)
trainer = pl.Trainer(gpus=1, max_epochs = 100)
trainer.fit(binaryClassifier, train_loader_diff, val_loader_diff)
torch.save(annModel.state_dict(), "./model/diff_ble")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | AnnModel          | 4.7 K 
1 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
4.7 K     Trainable params
0         Non-trainable params
4.7 K     Total params
0.019     Total estimated model params size (MB)
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/opt/conda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


In [13]:
annModel = pl_module.AnnModel(input_size = 80)
binaryClassifier = pl_module.BinaryClassifier(annModel)
trainer = pl.Trainer(gpus=1, max_epochs = 100)
trainer.fit(binaryClassifier, train_loader_each, val_loader_each)
torch.save(annModel.state_dict(), "./model/each_ble")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | AnnModel          | 7.3 K 
1 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
7.3 K     Trainable params
0         Non-trainable params
7.3 K     Total params
0.029     Total estimated model params size (MB)


# Load Wifi data

In [14]:
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data.dataset import random_split

x_tensor_diff = torch.from_numpy(np.concatenate((same_diff_wifi, diff_diff_wifi))).float()
x_tensor_each = torch.from_numpy(np.concatenate((same_each_wifi, diff_each_wifi))).float()

y = np.concatenate((np.zeros([len(same_each_wifi), 1]) + 1, np.zeros([len(diff_each_wifi), 1])))
y_tensor = torch.from_numpy(y).float()

dataset_diff = TensorDataset(x_tensor_diff, y_tensor)
dataset_each = TensorDataset(x_tensor_each, y_tensor)

train_len_diff = len(dataset_diff) // 5 * 4
train_len_each = len(dataset_each) // 5 * 4
val_len_diff = len(dataset_diff) - train_len_diff
val_len_each  = len(dataset_each) - train_len_each

train_dataset_diff, val_dataset_diff = random_split(dataset_diff, [train_len_diff, val_len_diff])
train_dataset_each, val_dataset_each = random_split(dataset_each, [train_len_each, val_len_each])

train_loader_diff = DataLoader(dataset=train_dataset_diff, batch_size=32, shuffle=True)
train_loader_each = DataLoader(dataset=train_dataset_each, batch_size=32, shuffle=True)
val_loader_diff = DataLoader(dataset = val_dataset_diff, batch_size = val_len_diff)
val_loader_each = DataLoader(dataset = val_dataset_each, batch_size = val_len_each)


In [15]:
annModel = pl_module.AnnModel(input_size = 40)
binaryClassifier = pl_module.BinaryClassifier(annModel)
trainer = pl.Trainer(gpus=1, max_epochs = 100)
trainer.fit(binaryClassifier, train_loader_diff, val_loader_diff)
torch.save(annModel.state_dict(), "./model/diff_wifi")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | AnnModel          | 4.7 K 
1 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
4.7 K     Trainable params
0         Non-trainable params
4.7 K     Total params
0.019     Total estimated model params size (MB)


In [16]:
annModel = pl_module.AnnModel(input_size = 80)
binaryClassifier = pl_module.BinaryClassifier(annModel)
trainer = pl.Trainer(gpus=1, max_epochs = 100)
trainer.fit(binaryClassifier, train_loader_each, val_loader_each)
torch.save(annModel.state_dict(), "./model/each_wifi")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | AnnModel          | 7.3 K 
1 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
7.3 K     Trainable params
0         Non-trainable params
7.3 K     Total params
0.029     Total estimated model params size (MB)
